In [2]:
import numpy as np
import pandas as pd
import PIL as PIL
import tensorflow as tf
import matplotlib.pyplot as plt
import tensorflow
import joblib

# model=joblib.load('res_net_without_augmentation_model.joblib')

## functions

In [10]:
def load(path,size=(120,120)):
    from tensorflow.keras.preprocessing.image import ImageDataGenerator
    training_dir = path
    training_generator = ImageDataGenerator(rescale=1/255)
    data= training_generator.flow_from_directory(training_dir,target_size=size,batch_size=8,class_mode="binary")
    train_x=[]
    train_y=[]
    for i in range(len(data)):
        image,label=data[i]
        train_x.extend(image)
        train_y.extend(label)
    return np.array(train_x),np.array(train_y)


In [11]:
def evaluate_my(model,x,y,th=0.5,name='evaluation: \n'):
    from sklearn.metrics import accuracy_score,recall_score,precision_score,f1_score
    predictions = model.predict(np.array(x))
    pred=(predictions>th).astype(int)
    print(f'accuracy:{accuracy_score(np.array(y),pred)}')
    print(f'recall:{recall_score(np.array(y),pred)}')
    print(f'precision:{precision_score(np.array(y),pred)}')
    print(f'f1_score:{f1_score(np.array(y),pred)}')

    

In [12]:
x_val,y_val=load('val',(220,220))
x_train,y_train=load('train',(220,220))
x_test,y_test=load('test',(220,220))

Found 26 images belonging to 2 classes.
Found 5206 images belonging to 2 classes.
Found 624 images belonging to 2 classes.


In [13]:
# from tensorflow.keras.preprocessing.image import ImageDataGenerator
# datagen = ImageDataGenerator(
#     zoom_range=0.2,
#     # horizontal_flip=True,
#     rotation_range=20,
#     width_shift_range=0.2,
#     height_shift_range=0.2,
# )
# # Create a generator for training data
# train_generator = datagen.flow(x_train, y_train, batch_size=32)

## normal model


In [16]:
model = tf.keras.Sequential([tf.keras.layers.Conv2D(32,(3,3),input_shape=(120,120,3),activation='relu'),
                            tf.keras.layers.MaxPooling2D(2,2),
                            tf.keras.layers.Conv2D(64,(3,3),activation='relu'),
                            tf.keras.layers.MaxPooling2D(2,2),
                            tf.keras.layers.Conv2D(128,(3,3),activation='relu'),
                            tf.keras.layers.MaxPooling2D(2,2),
                            tf.keras.layers.Conv2D(256,(3,3),activation='relu'),
                            tf.keras.layers.MaxPooling2D(2,2),
                            tf.keras.layers.Conv2D(512,(3,3),activation='relu'),
                            tf.keras.layers.MaxPooling2D(2,2),
                            tf.keras.layers.Flatten(),
                            tf.keras.layers.Dense(256,activation="relu"),
                            tf.keras.layers.Dense(1,activation="sigmoid")]) 
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),loss="binary_crossentropy",metrics=["acc"])

In [18]:
model.fit(np.array(x_train),np.array(y_train),epochs=2,validation_data=[np.array(x_val),np.array(y_val)])
# model.save('normal_model.h5')

Epoch 1/2


ValueError: Exception encountered when calling Sequential.call().

[1mInput 0 of layer "dense_2" is incompatible with the layer: expected axis -1 of input shape to have value 512, but received input with shape (None, 8192)[0m

Arguments received by Sequential.call():
  • inputs=tf.Tensor(shape=(None, 220, 220, 3), dtype=float32)
  • training=True
  • mask=None

In [ ]:
evaluate_my(model,x_test,y_test)
evaluate_my(model,x_train,y_train)
evaluate_my(model,x_val,y_val)

In [ ]:
# 20/20 [==============================] - 2s 108ms/step
# accuracy:0.7756410256410257
# recall:0.9871794871794872
# precision:0.7403846153846154
# f1_score:0.8461538461538463
# 163/163 [==============================] - 22s 131ms/step
# accuracy:0.9892638036809815
# recall:0.9891612903225806
# precision:0.9963608006238628
# f1_score:0.9927479927479927
# 1/1 [==============================] - 0s 105ms/step
# accuracy:0.9375
# recall:1.0
# precision:0.8888888888888888
# f1_score:0.9411764705882353

## alex net

In [19]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, BatchNormalization, Dropout

# Assuming you have 3 classes in your dataset
num_classes = 2

# Define the input shape
input_shape = (227, 227, 3)  # AlexNet default input size

# Create the AlexNet model
model = Sequential()

# Layer 1: Convolutional layer with 96 filters, 11x11 kernel size, and stride of 4x4
model.add(Conv2D(96, (11, 11), strides=(4, 4), activation='relu', input_shape=input_shape))

# Layer 2: MaxPooling layer with 3x3 pool size and stride of 2x2
model.add(MaxPooling2D((3, 3), strides=(2, 2)))

# Layer 3: Convolutional layer with 256 filters, 5x5 kernel size
model.add(Conv2D(256, (5, 5), activation='relu'))

# Layer 4: MaxPooling layer with 3x3 pool size and stride of 2x2
model.add(MaxPooling2D((3, 3), strides=(2, 2)))

# Layer 5: Convolutional layer with 384 filters, 3x3 kernel size
model.add(Conv2D(384, (3, 3), activation='relu'))

# Layer 6: Convolutional layer with 384 filters, 3x3 kernel size
model.add(Conv2D(384, (3, 3), activation='relu'))

# Layer 7: Convolutional layer with 256 filters, 3x3 kernel size
model.add(Conv2D(256, (3, 3), activation='relu'))

# Layer 8: MaxPooling layer with 3x3 pool size and stride of 2x2
model.add(MaxPooling2D((3, 3), strides=(2, 2)))

# Flatten the output for the fully connected layers
model.add(Flatten())

# Layer 9: Fully connected layer with 4096 units
model.add(Dense(4096, activation='relu'))

# Add Batch Normalization and Dropout for regularization
model.add(BatchNormalization())
model.add(Dropout(0.5))

# Layer 10: Fully connected layer with 4096 units
model.add(Dense(4096, activation='relu'))

# Add Batch Normalization and Dropout for regularization
model.add(BatchNormalization())
model.add(Dropout(0.5))

# Layer 11: Fully connected layer with the number of units equal to the number of classes
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [20]:
x_train_re=resize(x_train,(227,227))
x_val_re=resize(x_val,(227,227))
x_test_re=resize(x_test,(227,227))

NameError: name 'resize' is not defined

In [ ]:
model.fit(x_train_re,np.array(y_train),epochs=2,validation_data=[x_val_re,np.array(y_val)])

In [ ]:
# evaluate_my(model,x_val_re,y_val)
evaluate_my(model,x_test_re,y_test)

In [ ]:
# val_score
# 1/1 [==============================] - 0s 116ms/step
# accuracy:0.5
# recall:0.0
# precision:0.0
# f1_score:0.0

## resnet50

In [21]:
from tensorflow.keras.applications import ResNet50V2
from tensorflow.keras.preprocessing import image
# from tensorflow.keras.applications.resnet50V2 import preprocess_input, decode_predictions
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D,Dropout, MaxPooling2D,Conv2D, Activation, Flatten 
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import SGD

# Load the pre-trained ResNet50 model without the top (fully connected) layers
base_model = ResNet50V2(weights='imagenet', include_top=False,input_shape=(220,220,3))

# Add a custom top (fully connected) layer for your specific task
x = base_model.output
x = Flatten()(x)
x = Dense(128, activation='relu')(x)
x=  Dropout(0.5)(x)
predictions = Dense(1, activation='sigmoid')(x)  # Adjust 'num_classes' based on your task

# Create the final model
model = Model(inputs=base_model.input, outputs=predictions)

for layer in base_model.layers:
    layer.trainable = False

# Freeze the weights of the pre-trained layers

# Compile the model with an appropriate optimizer, loss, and metrics
model.compile(optimizer='adam', loss='binary_crossentropy',metrics=['accuracy'])

# Now, you can train the model on your specific dataset using model.fit()
# Make sure to preprocess your input data appropriately, e.g., using preprocess_input


In [22]:
model.fit(x_train,y_train,epochs=10,batch_size=16)

Epoch 1/10
326/326 ━━━━━━━━━━━━━━━━━━━━ 338s 966ms/step - accuracy: 0.8874 - loss: 1.4193
Epoch 2/10
326/326 ━━━━━━━━━━━━━━━━━━━━ 280s 854ms/step - accuracy: 0.9542 - loss: 0.1591
Epoch 3/10
 93/326 ━━━━━━━━━━━━━━━━━━━━ 3:14 835ms/step - accuracy: 0.9664 - loss: 0.1357

KeyboardInterrupt: 

In [8]:
model.save('resnet50_with_drop.model')

INFO:tensorflow:Assets written to: resnet50_with_drop.model\assets


INFO:tensorflow:Assets written to: resnet50_with_drop.model\assets


In [ ]:

evaluate_my('val',model,x_val,y_val)
evaluate_my('train',model,x_train,y_train)

evaluate_my('test',model,x_test,y_test)

In [ ]:
# for more then 15 epoch accuracy starts decresing

# for augmented(15 epochs, batch=32,th=0.5)

# val:
# 1/1 [==============================] - 1s 1s/step
# accuracy:0.875
# recall:1.0
# precision:0.8
# f1_score:0.888888888888889

# train:
# 163/163 [==============================] - 13s 78ms/step
# accuracy:0.9578220858895705
# recall:0.9483870967741935
# precision:0.9945872801082544
# f1_score:0.9709379128137384

# test:
# 20/20 [==============================] - 2s 77ms/step
# accuracy:0.9150641025641025
# recall:0.9615384615384616
# precision:0.9079903147699758
# f1_score:0.933997509339975


# for th=0.6

# 163/163 [==============================] - 8s 51ms/step
# accuracy:0.9451687116564417
# recall:0.9300645161290323
# precision:0.995855208621166
# f1_score:0.9618361355751267
# 20/20 [==============================] - 1s 50ms/step
# accuracy:0.9214743589743589
# recall:0.9564102564102565
# precision:0.9209876543209876
# f1_score:0.9383647798742139



# best accuracy for resnet_model with extra 64 layer(at th=0.7)
# 163/163 [==============================] - 8s 51ms/step
# accuracy:0.9392254601226994
# recall:0.9228387096774193
# precision:0.994991652754591
# f1_score:0.9575579060115142
# 20/20 [==============================] - 1s 50ms/step
# accuracy:0.9230769230769231
# recall:0.9564102564102565
# precision:0.9232673267326733
# f1_score:0.9395465994962218



# for augmented model(RESNET,10 epochs,32)
# accuracy

#val
# 1/1 [==============================] - 1s 828ms/step
# accuracy:0.9375
# recall:1.0
# precision:0.8888888888888888
# f1_score:0.9411764705882353
# 163/163 [==============================] - 8s 51ms/step

#train

# accuracy:0.9545628834355828
# recall:0.944258064516129
# precision:0.9942934782608696
# f1_score:0.9686300463269358
# 20/20 [==============================] - 1s 50ms/step

#test

# accuracy:0.907051282051282
# recall:0.9615384615384616
# precision:0.8971291866028708
# f1_score:0.9282178217821783



# #for non-augmented model(resnet)
# # val
# 1/1 [==============================] - 1s 903ms/step
# accuracy:0.9375
# recall:1.0
# precision:0.8888888888888888
# f1_score:0.9411764705882353

# # train
# 163/163 [==============================] - 9s 51ms/step
# accuracy:0.9975076687116564
# recall:0.9966451612903225
# precision:1.0
# f1_score:0.9983197621817242

# #test
# 20/20 [==============================] - 1s 50ms/step
# accuracy:0.8413461538461539
# recall:0.9948717948717949
# precision:0.8
# f1_score:0.8868571428571429

## applying mobilenet

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.applications.mobilenet import preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator

base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(220, 220, 3))

# Freeze the pre-trained layers
for layer in base_model.layers:
    layer.trainable = False

# Add a global spatial average pooling layer
x = base_model.output
x = GlobalAveragePooling2D()(x)

# Add a dense layer for classification
x = Dense(128, activation='relu')(x)

# Add the output layer with sigmoid activation for binary classification
predictions = Dense(1, activation='sigmoid')(x)

# Create the model
model = Model(inputs=base_model.input, outputs=predictions)

# Compile the model
model.compile(optimizer=Adam(lr=0.001), loss=tf.keras.losses.BinaryCrossentropy(from_logits=True), metrics=['accuracy'])

In [ ]:
epochs = 15
model.fit(
    train_generator,
    epochs=epochs,
)

In [ ]:
evaluate_my(model,x_val,y_val)
evaluate_my(model,x_train,y_train)
evaluate_my(model,x_test,y_test)

In [ ]:
# for mobile netv2 augmented model(th=0.5)

# 1/1 [==============================] - 1s 1s/step
# accuracy:0.9375
# recall:1.0
# precision:0.8888888888888888
# f1_score:0.9411764705882353
# 163/163 [==============================] - 4s 26ms/step
# accuracy:0.963957055214724
# recall:0.9636129032258065
# precision:0.9875694260777572
# f1_score:0.9754440961337514
# 20/20 [==============================] - 1s 25ms/step
# accuracy:0.8862179487179487
# recall:0.9871794871794872
# precision:0.8536585365853658
# f1_score:0.915576694411415

# we tried diffrent threshholds but resnet was better


## best accuracy:

In [ ]:
# for th=0.6 in resnet50v2_augmented

# 163/163 [==============================] - 8s 51ms/step
# accuracy:0.9451687116564417
# recall:0.9300645161290323
# precision:0.995855208621166
# f1_score:0.9618361355751267
# 20/20 [==============================] - 1s 50ms/step
# accuracy:0.9214743589743589
# recall:0.9564102564102565
# precision:0.9209876543209876
# f1_score:0.9383647798742139
